In [36]:
import pandas as pd
import csv
import requests
import csv
from lxml import html
import re
from random import randint
import time
from datetime import datetime,date

headersx = {
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,en-GB;q=0.8",
"cache-control": "no-cache",
"cookie": "TMSESSID=49unp8sbp99moc95uhqljv0ob4; utag_main=v_id:0163d8d03800001c6172f1bd893304069002006100868$_sn:3$_ss:0$_st:1528450968811$dc_visit_transfermarkt-transfermarkt.de:3$ses_id:1528446062028%3Bexp-session$_pn:18%3Bexp-session$dc_event_transfermarkt-transfermarkt.de:18%3Bexp-session",
"pragma": "no-cache",
"referer": "https://www.transfermarkt.co.uk",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
}

player_df = pd.read_csv('data/fifa_player_list.csv')

#code for finding age from https://stackoverflow.com/questions/2217488/age-from-birthdate-in-python
today = date.today()

labels = ['index_number','country','shirt_number','fifa_name','row_player_url','row_player_url_deeper']

with open("data/player_url_store.csv", "w") as filex:   
    wr = csv.writer(filex, delimiter = ',' , quotechar = '"' )
    wr.writerow(labels)

for index,row in player_df.iterrows():
    
    if index > 654: #for checking particular rows
    
        index_number = row['index']
        print(index_number)

        country = row['country']
        shirt_number = row['shirt_number']
        fifa_name = row['fifa_display_name']

        date_birth_raw = row['date_of_birth']
        date_birth = datetime.strptime(date_birth_raw,'%d.%m.%Y')
        age = today.year - date_birth.year - ((today.month, today.day) < (date_birth.month, date_birth.day))

        age_list = [age-1,age,age+1]

        fifa_name_lower = fifa_name.lower()
        query_fragment = fifa_name_lower.replace(" ","+") 

        fifa_name_part_list = fifa_name_lower.split()

        urlx = "https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=" +\
                            query_fragment + "&x=0&y=0"

        print(urlx)

        responsex = requests.get(urlx, headers = headersx)

        treex = html.fromstring(responsex.content)

        try:
            treex.xpath("//*[@id='main']/div[8]/div/div[1]/div[1]/h2/text()")[0]

        except:

            tablex = treex.xpath("//*[@id='yw0']/table")[0]

            row_range = tablex.xpath(".//descendant::tr[@class='odd' or @class='even']")

    #         row_range = tablex.xpath("//descendant::td[@class='hauptlink']/parent::tr]")

            possible_players = 0

            for row in row_range:

    #                 try:
    #                     row_player_url
    #                 except:
    #                     pass
    #                 else:
    #                     del row_player_url

    #                 try:
    #                     row_player_url_deeper
    #                 except:
    #                     pass
    #                 else:
    #                     del row_player_url_deeper

                row_player_age_string = row.xpath("td[@class='zentriert'][3]/text()")[0]

                try:
                    int(row_player_age_string)
                except:
                    continue
                else:
                    row_player_age = int(row_player_age_string)

    #             row_player_age_list = [row_player_age-1,row_player_age,row_player_age+1]

                row_country_list = row.xpath("td[@class='zentriert'][4]//img[@class='flaggenrahmen']/@title")

                row_player_url_fragmentx = row.xpath(".//td[@class='hauptlink']/a/@href")[0]

                row_player_name_fragmentx = (re.findall(r'/(.*)/profil/spieler/.*', row_player_url_fragmentx))[0] 

                row_player_name_part_list = row_player_name_fragmentx.split('-')

                if ((row_player_age in age_list) and\
                    (country in row_country_list) and\
                    (all(elem in row_player_name_part_list for elem in fifa_name_part_list))):

                    possible_players += 1

                    row_player_url_fragment = row.xpath(".//td[@class='hauptlink']/a/@href")[0]

                    row_player_url = "https://www.transfermarkt.co.uk" + row_player_url_fragment
    #                 print(row_player_url)

                    row_player_url_deeper = row_player_url.replace('profil','leistungsdatendetails')

                else:
                    continue
    #                     print('am at this point')

            if possible_players == 0:
                print("am getting results, but no matches for '{}'".format(fifa_name))

            elif possible_players > 1: 
                print("am getting multiple matches for '{}'".format(fifa_name))

            elif possible_players == 1:
                print("am getting one match for '{}'".format(fifa_name))

                new_row = [index_number,country,shirt_number,fifa_name,row_player_url,row_player_url_deeper]

                with open("data/player_url_store.csv", "a") as filez:
                    wrz = csv.writer(filez, delimiter = ',' , quotechar = '"' )
                    wrz.writerow(new_row)

        else:

            no_result_signal = treex.xpath("//*[@id='main']/div[8]/div/div[1]/div[1]/h2/text()")[0]

            if no_result_signal == "Nothing found?":
                print("no results found for '{}'".format(fifa_name))
                continue
            else:
                print("dont know why I'm getting this print statement for {}".format(fifa_name))

        delay = randint(2,4)
        time.sleep(delay)       
    
print('done and done')     

# 353 stopped here
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ju+sejong&x=0&y=0


#how do you make sure variables dont get carried between loops?

656
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=karl-johan+johnsson&x=0&y=0
am getting results, but no matches for 'Karl-Johan JOHNSSON'
657
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gustav+svensson&x=0&y=0
am getting one match for 'Gustav SVENSSON'
658
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=filip+helander&x=0&y=0
am getting one match for 'Filip HELANDER'
659
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oscar+hiljemark&x=0&y=0
am getting one match for 'Oscar HILJEMARK'
660
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=emil+krafth&x=0&y=0
am getting one match for 'Emil KRAFTH'
661
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=viktor+claesson&x=0&y=0
am getting one match for 'Viktor CLAESSON'
662
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pontus+jansson&x=0&y=0
am getting one match for

am getting one match for 'Naim SLITI'
714
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fernando+muslera&x=0&y=0
am getting one match for 'Fernando MUSLERA'
715
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+gimenez&x=0&y=0
am getting one match for 'Jose GIMENEZ'
716
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=diego+godin&x=0&y=0
am getting one match for 'Diego GODIN'
717
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=guillermo+varela&x=0&y=0
am getting one match for 'Guillermo VARELA'
718
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+sanchez&x=0&y=0
am getting one match for 'Carlos SANCHEZ'
719
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rodrigo+bentancur&x=0&y=0
am getting one match for 'Rodrigo BENTANCUR'
720
https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristian+rodriguez&x=0&y=0
am

In [ ]:
# 1
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nahuel+guzman&x=0&y=0
# am getting one match for 'Nahuel GUZMAN'
# 2
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gabriel+mercado&x=0&y=0
# am getting one match for 'Gabriel MERCADO'
# 3
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nicolas+tagliafico&x=0&y=0
# am getting one match for 'Nicolas TAGLIAFICO'
# 4
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristian+ansaldi&x=0&y=0
# am getting one match for 'Cristian ANSALDI'
# 5
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lucas+biglia&x=0&y=0
# am getting one match for 'Lucas BIGLIA'
# 6
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=federico+fazio&x=0&y=0
# am getting one match for 'Federico FAZIO'
# 7
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ever+banega&x=0&y=0
# am getting one match for 'Ever BANEGA'
# 8
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcos+acuna&x=0&y=0
# am getting one match for 'Marcos ACUNA'
# 9
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gonzalo+higuain&x=0&y=0
# am getting one match for 'Gonzalo HIGUAIN'
# 10
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lionel+messi&x=0&y=0
# am getting one match for 'Lionel MESSI'
# 11
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=angel+di+maria&x=0&y=0
# am getting one match for 'Angel DI MARIA'
# 12
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=franco+armani&x=0&y=0
# am getting one match for 'Franco ARMANI'
# 13
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=maximiliano+meza&x=0&y=0
# am getting one match for 'Maximiliano MEZA'
# 14
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=javier+mascherano&x=0&y=0
# am getting one match for 'Javier MASCHERANO'
# 15
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=manuel+lanzini&x=0&y=0
# am getting one match for 'Manuel LANZINI'
# 16
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcos+rojo&x=0&y=0
# am getting one match for 'Marcos ROJO'
# 17
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nicolas+otamendi&x=0&y=0
# am getting one match for 'Nicolas OTAMENDI'
# 18
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=eduardo+salvio&x=0&y=0
# am getting one match for 'Eduardo SALVIO'
# 19
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sergio+aguero&x=0&y=0
# am getting one match for 'Sergio AGUERO'
# 20
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=giovani+lo+celso&x=0&y=0
# am getting one match for 'Giovani LO CELSO'
# 21
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=paulo+dybala&x=0&y=0
# am getting one match for 'Paulo DYBALA'
# 22
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristian+pavon&x=0&y=0
# am getting one match for 'Cristian PAVON'
# 23
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=wilfredo+caballero&x=0&y=0
# am getting results, but no matches for 'Wilfredo CABALLERO'
# 24
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mathew+ryan&x=0&y=0
# am getting one match for 'Mathew RYAN'
# 25
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=milos+degenek&x=0&y=0
# am getting one match for 'Milos DEGENEK'
# 26
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=james+meredith&x=0&y=0
# am getting one match for 'James MEREDITH'
# 27
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tim+cahill&x=0&y=0
# am getting one match for 'Tim CAHILL'
# 28
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mark+milligan&x=0&y=0
# am getting one match for 'Mark MILLIGAN'
# 29
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=matthew+jurman&x=0&y=0
# am getting one match for 'Matthew JURMAN'
# 30
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mathew+leckie&x=0&y=0
# am getting one match for 'Mathew LECKIE'
# 31
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=massimo+luongo&x=0&y=0
# am getting one match for 'Massimo LUONGO'
# 32
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tomi+juric&x=0&y=0
# am getting one match for 'Tomi JURIC'
# 33
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=robbie+kruse&x=0&y=0
# am getting one match for 'Robbie KRUSE'
# 34
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andrew+nabbout&x=0&y=0
# am getting one match for 'Andrew NABBOUT'
# 35
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=brad+jones&x=0&y=0
# am getting one match for 'Brad JONES'
# 36
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aaron+mooy&x=0&y=0
# am getting one match for 'Aaron MOOY'
# 37
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jamie+maclaren&x=0&y=0
# am getting one match for 'Jamie MacLAREN'
# 38
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mile+jedinak&x=0&y=0
# am getting one match for 'Mile JEDINAK'
# 39
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aziz+behich&x=0&y=0
# am getting one match for 'Aziz BEHICH'
# 40
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=daniel+arzani&x=0&y=0
# am getting one match for 'Daniel ARZANI'
# 41
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=danny+vukovic&x=0&y=0
# am getting one match for 'Danny VUKOVIC'
# 42
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=joshua+risdon&x=0&y=0
# am getting one match for 'Joshua RISDON'
# 43
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=trent+sainsbury&x=0&y=0
# am getting one match for 'Trent SAINSBURY'
# 44
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dimitrios+petratos&x=0&y=0
# am getting results, but no matches for 'Dimitrios PETRATOS'
# 45
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jackson+irvine&x=0&y=0
# am getting one match for 'Jackson IRVINE'
# 46
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tom+rogic&x=0&y=0
# am getting one match for 'Tom ROGIC'
# 47
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thibaut+courtois&x=0&y=0
# am getting one match for 'Thibaut COURTOIS'
# 48
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=toby+alderweireld&x=0&y=0
# am getting one match for 'Toby ALDERWEIRELD'
# 49
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thomas+vermaelen&x=0&y=0
# am getting one match for 'Thomas VERMAELEN'
# 50
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=vincent+kompany&x=0&y=0
# am getting one match for 'Vincent KOMPANY'
# 51
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jan+vertonghen&x=0&y=0
# am getting one match for 'Jan VERTONGHEN'
# 52
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=axel+witsel&x=0&y=0
# am getting one match for 'Axel WITSEL'
# 53
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kevin+de+bruyne&x=0&y=0
# am getting one match for 'Kevin DE BRUYNE'
# 54
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marouane+fellaini&x=0&y=0
# am getting one match for 'Marouane FELLAINI'
# 55
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=romelu+lukaku&x=0&y=0
# am getting one match for 'Romelu LUKAKU'
# 56
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=eden+hazard&x=0&y=0
# am getting one match for 'Eden HAZARD'
# 57
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yannick+carrasco&x=0&y=0
# am getting one match for 'Yannick CARRASCO'
# 58
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=simon+mignolet&x=0&y=0
# am getting one match for 'Simon MIGNOLET'
# 59
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=koen+casteels&x=0&y=0
# am getting one match for 'Koen CASTEELS'
# 60
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dries+mertens&x=0&y=0
# am getting one match for 'Dries MERTENS'
# 61
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thomas+meunier&x=0&y=0
# am getting one match for 'Thomas MEUNIER'
# 62
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thorgan+hazard&x=0&y=0
# am getting one match for 'Thorgan HAZARD'
# 63
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=youri+tielemans&x=0&y=0
# am getting one match for 'Youri TIELEMANS'
# 64
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=adnan+januzaj&x=0&y=0
# am getting one match for 'Adnan JANUZAJ'
# 65
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=moussa+dembele&x=0&y=0
# am getting results, but no matches for 'Moussa DEMBELE'
# 66
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dedryck+boyata&x=0&y=0
# am getting one match for 'Dedryck BOYATA'
# 67
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=michy+batshuayi&x=0&y=0
# am getting one match for 'Michy BATSHUAYI'
# 68
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nacer+chadli&x=0&y=0
# am getting one match for 'Nacer CHADLI'
# 69
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=leander+dendoncker&x=0&y=0
# am getting one match for 'Leander DENDONCKER'
# 70
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alisson&x=0&y=0
# am getting multiple matches for 'ALISSON'
# 71
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thiago+silva&x=0&y=0
# am getting multiple matches for 'THIAGO SILVA'
# 72
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=miranda&x=0&y=0
# am getting one match for 'MIRANDA'
# 73
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pedro+geromel&x=0&y=0
# am getting one match for 'PEDRO GEROMEL'
# 74
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=casemiro&x=0&y=0
# am getting one match for 'CASEMIRO'
# 75
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=filipe+luis&x=0&y=0
# am getting one match for 'FILIPE LUIS'
# 76
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=douglas+costa&x=0&y=0
# am getting one match for 'DOUGLAS COSTA'
# 77
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=renato+augusto&x=0&y=0
# am getting one match for 'RENATO AUGUSTO'
# 78
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gabriel+jesus&x=0&y=0
# am getting one match for 'GABRIEL JESUS'
# 79
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=neymar&x=0&y=0
# am getting one match for 'NEYMAR'
# 80
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=philippe+coutinho&x=0&y=0
# am getting one match for 'PHILIPPE COUTINHO'
# 81
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcelo&x=0&y=0
# am getting multiple matches for 'MARCELO'
# 82
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marquinhos&x=0&y=0
# am getting one match for 'MARQUINHOS'
# 83
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=danilo&x=0&y=0
# am getting one match for 'DANILO'
# 84
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=paulinho&x=0&y=0
# am getting multiple matches for 'PAULINHO'
# 85
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cassio&x=0&y=0
# am getting multiple matches for 'CASSIO'
# 86
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fernandinho&x=0&y=0
# am getting multiple matches for 'FERNANDINHO'
# 87
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fred&x=0&y=0
# am getting one match for 'FRED'
# 88
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=willian&x=0&y=0
# am getting multiple matches for 'WILLIAN'
# 89
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=roberto+firmino&x=0&y=0
# am getting one match for 'ROBERTO FIRMINO'
# 90
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=taison&x=0&y=0
# am getting one match for 'TAISON'
# 91
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fagner&x=0&y=0
# am getting multiple matches for 'FAGNER'
# 92
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ederson&x=0&y=0
# am getting one match for 'EDERSON'
# 93
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=david+ospina&x=0&y=0
# am getting one match for 'David OSPINA'
# 94
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristian+zapata&x=0&y=0
# am getting one match for 'Cristian ZAPATA'
# 95
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oscar+murillo&x=0&y=0
# am getting one match for 'Oscar MURILLO'
# 96
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=santiago+arias&x=0&y=0
# am getting one match for 'Santiago ARIAS'
# 97
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=wilmar+barrios&x=0&y=0
# am getting one match for 'Wilmar BARRIOS'
# 98
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+sanchez&x=0&y=0
# am getting one match for 'Carlos SANCHEZ'
# 99
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+bacca&x=0&y=0
# am getting one match for 'Carlos BACCA'
# 100
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abel+aguilar&x=0&y=0
# am getting one match for 'Abel AGUILAR'
# 101
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=radamel+falcao&x=0&y=0
# am getting results, but no matches for 'Radamel FALCAO'
# 102
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=james+rodriguez&x=0&y=0
# am getting one match for 'James RODRIGUEZ'
# 103
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=juan+cuadrado&x=0&y=0
# am getting one match for 'Juan CUADRADO'
# 104
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=camilo+vargas&x=0&y=0
# am getting one match for 'Camilo VARGAS'
# 105
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yerry+mina&x=0&y=0
# am getting one match for 'Yerry MINA'
# 106
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luis+muriel&x=0&y=0
# am getting one match for 'Luis MURIEL'
# 107
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mateus+uribe&x=0&y=0
# am getting one match for 'Mateus URIBE'
# 108
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jefferson+lerma&x=0&y=0
# am getting one match for 'Jefferson LERMA'
# 109
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=johan+mojica&x=0&y=0
# am getting one match for 'Johan MOJICA'
# 110
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=frank+fabra&x=0&y=0
# am getting one match for 'Frank FABRA'
# 111
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=miguel+borja&x=0&y=0
# am getting one match for 'Miguel BORJA'
# 112
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=juan+quintero&x=0&y=0
# am getting one match for 'Juan QUINTERO'
# 113
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+izquierdo&x=0&y=0
# am getting one match for 'Jose IZQUIERDO'
# 114
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+cuadrado&x=0&y=0
# am getting one match for 'Jose CUADRADO'
# 115
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=davinson+sanchez&x=0&y=0
# am getting one match for 'Davinson SANCHEZ'
# 116
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=keylor+navas&x=0&y=0
# am getting one match for 'Keylor NAVAS'
# 117
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=johnny+acosta&x=0&y=0
# no results found for 'Johnny ACOSTA'
# 118
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=giancarlo+gonzalez&x=0&y=0
# am getting one match for 'Giancarlo GONZALEZ'
# 119
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ian+smith&x=0&y=0
# am getting one match for 'Ian SMITH'
# 120
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=celso+borges&x=0&y=0
# am getting one match for 'Celso BORGES'
# 121
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oscar+duarte&x=0&y=0
# am getting one match for 'Oscar DUARTE'
# 122
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=christian+bolanos&x=0&y=0
# am getting one match for 'Christian BOLANOS'
# 123
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bryan+oviedo&x=0&y=0
# am getting one match for 'Bryan OVIEDO'
# 124
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=daniel+colindres&x=0&y=0
# am getting one match for 'Daniel COLINDRES'
# 125
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bryan+ruiz&x=0&y=0
# am getting one match for 'Bryan RUIZ'
# 126
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=johan+venegas&x=0&y=0
# am getting one match for 'Johan VENEGAS'
# 127
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=joel+campbell&x=0&y=0
# am getting one match for 'Joel CAMPBELL'
# 128
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rodney+wallace&x=0&y=0
# am getting one match for 'Rodney WALLACE'
# 129
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=randall+azofeifa&x=0&y=0
# am getting one match for 'Randall AZOFEIFA'
# 130
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=francisco+calvo&x=0&y=0
# am getting one match for 'Francisco CALVO'
# 131
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristian+gamboa&x=0&y=0
# am getting one match for 'Cristian GAMBOA'
# 132
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yeltsin+tejeda&x=0&y=0
# am getting one match for 'Yeltsin TEJEDA'
# 133
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=patrick+pemberton&x=0&y=0
# am getting one match for 'Patrick PEMBERTON'
# 134
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kendall+waston&x=0&y=0
# am getting one match for 'Kendall WASTON'
# 135
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=david+guzman&x=0&y=0
# am getting multiple matches for 'David GUZMAN'
# 136
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcos+urena&x=0&y=0
# am getting one match for 'Marcos URENA'
# 137
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ronald+matarrita&x=0&y=0
# am getting one match for 'Ronald MATARRITA'
# 138
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=leonel+moreira&x=0&y=0
# am getting one match for 'Leonel MOREIRA'
# 139
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dominik+livakovic&x=0&y=0
# am getting one match for 'Dominik LIVAKOVIC'
# 140
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sime+vrsaljko&x=0&y=0
# am getting one match for 'Sime VRSALJKO'
# 141
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ivan+strinic&x=0&y=0
# am getting one match for 'Ivan STRINIC'
# 142
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ivan+perisic&x=0&y=0
# am getting one match for 'Ivan PERISIC'
# 143
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=vedran+corluka&x=0&y=0
# am getting one match for 'Vedran CORLUKA'
# 144
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dejan+lovren&x=0&y=0
# am getting one match for 'Dejan LOVREN'
# 145
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ivan+rakitic&x=0&y=0
# am getting one match for 'Ivan RAKITIC'
# 146
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mateo+kovacic&x=0&y=0
# am getting multiple matches for 'Mateo KOVACIC'
# 147
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andrej+kramaric&x=0&y=0
# am getting one match for 'Andrej KRAMARIC'
# 148
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luka+modric&x=0&y=0
# am getting one match for 'Luka MODRIC'
# 149
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcelo+brozovic&x=0&y=0
# am getting one match for 'Marcelo BROZOVIC'
# 150
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lovre+kalinic&x=0&y=0
# am getting one match for 'Lovre KALINIC'
# 151
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tin+jedvaj&x=0&y=0
# am getting one match for 'Tin JEDVAJ'
# 152
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=filip+bradaric&x=0&y=0
# am getting one match for 'Filip BRADARIC'
# 153
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=duje+caleta-car&x=0&y=0
# am getting results, but no matches for 'Duje CALETA-CAR'
# 154
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nikola+kalinic&x=0&y=0
# am getting one match for 'Nikola KALINIC'
# 155
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mario+mandzukic&x=0&y=0
# am getting one match for 'Mario MANDZUKIC'
# 156
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ante+rebic&x=0&y=0
# am getting one match for 'Ante REBIC'
# 157
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=milan+badelj&x=0&y=0
# am getting one match for 'Milan BADELJ'
# 158
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marko+pjaca&x=0&y=0
# am getting one match for 'Marko PJACA'
# 159
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=domagoj+vida&x=0&y=0
# am getting one match for 'Domagoj VIDA'
# 160
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=josip+pivaric&x=0&y=0
# am getting one match for 'Josip PIVARIC'
# 161
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=danijel+subasic&x=0&y=0
# am getting one match for 'Danijel SUBASIC'
# 162
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kasper+schmeichel&x=0&y=0
# am getting one match for 'Kasper SCHMEICHEL'
# 163
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=michael+krohn-dehli&x=0&y=0
# am getting results, but no matches for 'Michael KROHN-DEHLI'
# 164
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jannik+vestergaard&x=0&y=0
# am getting one match for 'Jannik VESTERGAARD'
# 165
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=simon+kjaer&x=0&y=0
# no results found for 'Simon KJAER'
# 166
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jonas+knudsen&x=0&y=0
# am getting one match for 'Jonas KNUDSEN'
# 167
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andreas+christensen&x=0&y=0
# am getting one match for 'Andreas CHRISTENSEN'
# 168
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=william+kvist&x=0&y=0
# am getting one match for 'William KVIST'
# 169
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thomas+delaney&x=0&y=0
# am getting one match for 'Thomas DELANEY'
# 170
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nicolai+jorgensen&x=0&y=0
# am getting one match for 'Nicolai JORGENSEN'
# 171
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=christian+eriksen&x=0&y=0
# am getting one match for 'Christian ERIKSEN'
# 172
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=martin+braithwaite&x=0&y=0
# am getting one match for 'Martin BRAITHWAITE'
# 173
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kasper+dolberg&x=0&y=0
# am getting one match for 'Kasper DOLBERG'
# 174
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mathias+jorgensen&x=0&y=0
# am getting results, but no matches for 'Mathias JORGENSEN'
# 175
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=henrik+dalsgaard&x=0&y=0
# am getting one match for 'Henrik DALSGAARD'
# 176
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=viktor+fischer&x=0&y=0
# am getting one match for 'Viktor FISCHER'
# 177
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jonas+lossl&x=0&y=0
# am getting one match for 'Jonas LOSSL'
# 178
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jens+stryger+larsen&x=0&y=0
# am getting one match for 'Jens STRYGER LARSEN'
# 179
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lukas+lerager&x=0&y=0
# am getting one match for 'Lukas LERAGER'
# 180
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lasse+schone&x=0&y=0
# am getting one match for 'Lasse SCHONE'
# 181
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yussuf+yurary+poulsen&x=0&y=0
# am getting results, but no matches for 'Yussuf Yurary POULSEN'
# 182
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andreas+cornelius&x=0&y=0
# am getting one match for 'Andreas CORNELIUS'
# 183
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=frederik+ronnow&x=0&y=0
# am getting one match for 'Frederik RONNOW'
# 184
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pione+sisto&x=0&y=0
# am getting one match for 'Pione SISTO'
# 185
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=essam+elhadary&x=0&y=0
# no results found for 'ESSAM ELHADARY'
# 186
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ali+gabr&x=0&y=0
# am getting one match for 'ALI GABR'
# 187
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ahmed+elmohamady&x=0&y=0
# am getting one match for 'AHMED ELMOHAMADY'
# 188
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=omar+gaber&x=0&y=0
# am getting one match for 'OMAR GABER'
# 189
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sam+morsy&x=0&y=0
# am getting one match for 'SAM MORSY'
# 190
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ahmed+hegazy&x=0&y=0
# am getting multiple matches for 'AHMED HEGAZY'
# 191
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ahmed+fathi&x=0&y=0
# am getting multiple matches for 'AHMED FATHI'
# 192
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tarek+hamed&x=0&y=0
# am getting one match for 'TAREK HAMED'
# 193
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marwan+mohsen&x=0&y=0
# am getting one match for 'MARWAN MOHSEN'
# 194
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohamed+salah&x=0&y=0
# am getting multiple matches for 'MOHAMED SALAH'
# 195
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kahraba&x=0&y=0
# am getting one match for 'KAHRABA'
# 196
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ayman+ashraf&x=0&y=0
# am getting one match for 'AYMAN ASHRAF'
# 197
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohamed+abdelshafy&x=0&y=0
# no results found for 'MOHAMED ABDELSHAFY'
# 198
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ramadan+sobhy&x=0&y=0
# no results found for 'RAMADAN SOBHY'
# 199
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mahmoud+hamdy&x=0&y=0
# am getting one match for 'MAHMOUD HAMDY'
# 200
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sherif+ekramy&x=0&y=0
# am getting one match for 'SHERIF EKRAMY'
# 201
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohamed+elneny&x=0&y=0
# am getting one match for 'MOHAMED ELNENY'
# 202
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=shikabala&x=0&y=0
# am getting one match for 'SHIKABALA'
# 203
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdalla+said&x=0&y=0
# no results found for 'ABDALLA SAID'
# 204
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=samir+saad&x=0&y=0
# am getting results, but no matches for 'SAMIR SAAD'
# 205
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=trezeguet&x=0&y=0
# am getting one match for 'TREZEGUET'
# 206
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=amr+warda&x=0&y=0
# am getting one match for 'AMR WARDA'
# 207
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohamed+elshenawy&x=0&y=0
# no results found for 'MOHAMED ELSHENAWY'
# 208
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jordan+pickford&x=0&y=0
# am getting one match for 'Jordan PICKFORD'
# 209
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kyle+walker&x=0&y=0
# am getting one match for 'Kyle WALKER'
# 210
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=danny+rose&x=0&y=0
# am getting one match for 'Danny ROSE'
# 211
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=eric+dier&x=0&y=0
# am getting one match for 'Eric DIER'
# 212
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=john+stones&x=0&y=0
# am getting one match for 'John STONES'
# 213
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=harry+maguire&x=0&y=0
# am getting one match for 'Harry MAGUIRE'
# 214
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jesse+lingard&x=0&y=0
# am getting one match for 'Jesse LINGARD'
# 215
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jordan+henderson&x=0&y=0
# am getting one match for 'Jordan HENDERSON'
# 216
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=harry+kane&x=0&y=0
# am getting one match for 'Harry KANE'
# 217
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=raheem+sterling&x=0&y=0
# am getting one match for 'Raheem STERLING'
# 218
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jamie+vardy&x=0&y=0
# am getting one match for 'Jamie VARDY'
# 219
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kieran+trippier&x=0&y=0
# am getting one match for 'Kieran TRIPPIER'
# 220
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jack+butland&x=0&y=0
# am getting one match for 'Jack BUTLAND'
# 221
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=danny+welbeck&x=0&y=0
# am getting one match for 'Danny WELBECK'
# 222
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gary+cahill&x=0&y=0
# am getting one match for 'Gary CAHILL'
# 223
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=phil+jones&x=0&y=0
# am getting one match for 'Phil JONES'
# 224
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fabian+delph&x=0&y=0
# am getting one match for 'Fabian DELPH'
# 225
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ashley+young&x=0&y=0
# am getting one match for 'Ashley YOUNG'
# 226
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcus+rashford&x=0&y=0
# am getting one match for 'Marcus RASHFORD'
# 227
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dele+alli&x=0&y=0
# am getting one match for 'Dele ALLI'
# 228
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ruben+loftus-cheek&x=0&y=0
# am getting results, but no matches for 'Ruben LOFTUS-CHEEK'
# 229
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=trent+alexander-arnold&x=0&y=0
# am getting results, but no matches for 'Trent ALEXANDER-ARNOLD'
# 230
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nick+pope&x=0&y=0
# am getting one match for 'Nick POPE'
# 231
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hugo+lloris&x=0&y=0
# am getting one match for 'Hugo LLORIS'
# 232
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=benjamin+pavard&x=0&y=0
# am getting one match for 'Benjamin PAVARD'
# 233
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=presnel+kimpembe&x=0&y=0
# am getting one match for 'Presnel KIMPEMBE'
# 234
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=raphael+varane&x=0&y=0
# am getting one match for 'Raphael VARANE'
# 235
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=samuel+umtiti&x=0&y=0
# am getting one match for 'Samuel UMTITI'
# 236
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=paul+pogba&x=0&y=0
# am getting one match for 'Paul POGBA'
# 237
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=antoine+griezmann&x=0&y=0
# am getting one match for 'Antoine GRIEZMANN'
# 238
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thomas+lemar&x=0&y=0
# am getting one match for 'Thomas LEMAR'
# 239
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=olivier+giroud&x=0&y=0
# am getting one match for 'Olivier GIROUD'
# 240
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kylian+mbappe&x=0&y=0
# am getting one match for 'Kylian MBAPPE'
# 241
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ousmane+dembele&x=0&y=0
# am getting one match for 'Ousmane DEMBELE'
# 242
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=corentin+tolisso&x=0&y=0
# am getting one match for 'Corentin TOLISSO'
# 243
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ngolo+kante&x=0&y=0
# no results found for 'Ngolo KANTE'
# 244
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=blaise+matuidi&x=0&y=0
# am getting one match for 'Blaise MATUIDI'
# 245
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=steven+nzonzi&x=0&y=0
# am getting one match for 'Steven NZONZI'
# 246
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=steve+mandanda&x=0&y=0
# am getting one match for 'Steve MANDANDA'
# 247
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=adil+rami&x=0&y=0
# am getting one match for 'Adil RAMI'
# 248
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nabil+fekir&x=0&y=0
# am getting one match for 'Nabil FEKIR'
# 249
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=djibril+sidibe&x=0&y=0
# am getting one match for 'Djibril SIDIBE'
# 250
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=florian+thauvin&x=0&y=0
# am getting one match for 'Florian THAUVIN'
# 251
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lucas+hernandez&x=0&y=0
# am getting one match for 'Lucas HERNANDEZ'
# 252
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=benjamin+mendy&x=0&y=0
# am getting one match for 'Benjamin MENDY'
# 253
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alphonse+areola&x=0&y=0
# am getting one match for 'Alphonse AREOLA'
# 254
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=manuel+neuer&x=0&y=0
# am getting one match for 'Manuel NEUER'
# 255
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marvin+plattenhardt&x=0&y=0
# am getting one match for 'Marvin PLATTENHARDT'
# 256
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jonas+hector&x=0&y=0
# am getting one match for 'Jonas HECTOR'
# 257
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=matthias+ginter&x=0&y=0
# am getting one match for 'Matthias GINTER'
# 258
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mats+hummels&x=0&y=0
# am getting one match for 'Mats HUMMELS'
# 259
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sami+khedira&x=0&y=0
# am getting one match for 'Sami KHEDIRA'
# 260
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=julian+draxler&x=0&y=0
# am getting one match for 'Julian DRAXLER'
# 261
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=toni+kroos&x=0&y=0
# am getting one match for 'Toni KROOS'
# 262
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=timo+werner&x=0&y=0
# am getting one match for 'Timo WERNER'
# 263
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mesut+oezil&x=0&y=0
# no results found for 'Mesut OEZIL'
# 264
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marco+reus&x=0&y=0
# am getting one match for 'Marco REUS'
# 265
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kevin+trapp&x=0&y=0
# am getting multiple matches for 'Kevin TRAPP'
# 266
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thomas+mueller&x=0&y=0
# am getting results, but no matches for 'Thomas MUELLER'
# 267
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=leon+goretzka&x=0&y=0
# am getting one match for 'Leon GORETZKA'
# 268
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=niklas+suele&x=0&y=0
# no results found for 'Niklas SUELE'
# 269
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=antonio+ruediger&x=0&y=0
# am getting results, but no matches for 'Antonio RUEDIGER'
# 270
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jerome+boateng&x=0&y=0
# am getting one match for 'Jerome BOATENG'
# 271
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=joshua+kimmich&x=0&y=0
# am getting one match for 'Joshua KIMMICH'
# 272
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sebastian+rudy&x=0&y=0
# am getting one match for 'Sebastian RUDY'
# 273
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=julian+brandt&x=0&y=0
# am getting one match for 'Julian BRANDT'
# 274
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ilkay+guendogan&x=0&y=0
# no results found for 'Ilkay GUENDOGAN'
# 275
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marc-andre+ter+stegen&x=0&y=0
# am getting results, but no matches for 'Marc-Andre TER STEGEN'
# 276
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mario+gomez&x=0&y=0
# am getting one match for 'Mario GOMEZ'
# 277
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hannes+halldorsson&x=0&y=0
# no results found for 'Hannes HALLDORSSON'
# 278
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=birkir+saevarsson&x=0&y=0
# no results found for 'Birkir SAEVARSSON'
# 279
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=samuel+fridjonsson&x=0&y=0
# am getting one match for 'Samuel FRIDJONSSON'
# 280
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=albert+gudmundsson&x=0&y=0
# am getting one match for 'Albert GUDMUNDSSON'
# 281
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sverrir+ingason&x=0&y=0
# no results found for 'Sverrir INGASON'
# 282
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ragnar+sigurdsson&x=0&y=0
# am getting one match for 'Ragnar SIGURDSSON'
# 283
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=johann+gudmundsson&x=0&y=0
# am getting results, but no matches for 'Johann GUDMUNDSSON'
# 284
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=birkir+bjarnason&x=0&y=0
# am getting one match for 'Birkir BJARNASON'
# 285
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bjorn+sigurdarson&x=0&y=0
# am getting one match for 'Bjorn SIGURDARSON'
# 286
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gylfi+sigurdsson&x=0&y=0
# am getting one match for 'Gylfi SIGURDSSON'
# 287
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alfred+finnbogason&x=0&y=0
# am getting one match for 'Alfred FINNBOGASON'
# 288
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=frederik+schram&x=0&y=0
# am getting one match for 'Frederik SCHRAM'
# 289
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=runar+runarsson&x=0&y=0
# no results found for 'Runar RUNARSSON'
# 290
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kari+arnason&x=0&y=0
# am getting one match for 'Kari ARNASON'
# 291
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=holmar+eyjolfsson&x=0&y=0
# no results found for 'Holmar EYJOLFSSON'
# 292
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=olafur+skulason&x=0&y=0
# no results found for 'Olafur SKULASON'
# 293
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aron+gunnarsson&x=0&y=0
# am getting one match for 'Aron GUNNARSSON'
# 294
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hordur+magnusson&x=0&y=0
# am getting one match for 'Hordur MAGNUSSON'
# 295
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rurik+gislason&x=0&y=0
# am getting one match for 'Rurik GISLASON'
# 296
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=emil+hallfredsson&x=0&y=0
# am getting one match for 'Emil HALLFREDSSON'
# 297
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=arnor+traustason&x=0&y=0
# no results found for 'Arnor TRAUSTASON'
# 298
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jon+bodvarsson&x=0&y=0
# no results found for 'Jon BODVARSSON'
# 299
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ari+skulason&x=0&y=0
# no results found for 'Ari SKULASON'
# 300
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ali+beiranvand&x=0&y=0
# no results found for 'Ali BEIRANVAND'
# 301
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mehdi+torabi&x=0&y=0
# am getting one match for 'Mehdi TORABI'
# 302
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ehsan+haji+safi&x=0&y=0
# no results found for 'Ehsan HAJI SAFI'
# 303
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=roozbeh+cheshmi&x=0&y=0
# am getting one match for 'Roozbeh CHESHMI'
# 304
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=milad+mohammadi&x=0&y=0
# am getting multiple matches for 'Milad MOHAMMADI'
# 305
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=saeid+ezatolahi&x=0&y=0
# am getting one match for 'Saeid EZATOLAHI'
# 306
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=masoud+shojaei&x=0&y=0
# am getting one match for 'Masoud SHOJAEI'
# 307
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=morteza+pouraliganji&x=0&y=0
# am getting one match for 'Morteza POURALIGANJI'
# 308
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=omid+ebrahimi&x=0&y=0
# am getting one match for 'Omid EBRAHIMI'
# 309
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=karim+ansarifard&x=0&y=0
# am getting one match for 'Karim ANSARIFARD'
# 310
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=vahid+amiri&x=0&y=0
# am getting one match for 'Vahid AMIRI'
# 311
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rashid+mazaheri&x=0&y=0
# am getting one match for 'Rashid MAZAHERI'
# 312
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohammad+reza+khanzadeh&x=0&y=0
# am getting one match for 'Mohammad Reza KHANZADEH'
# 313
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=saman+ghoddos&x=0&y=0
# am getting one match for 'Saman GHODDOS'
# 314
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pejman+montazeri&x=0&y=0
# am getting one match for 'Pejman MONTAZERI'
# 315
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=reza+ghoochannejhad&x=0&y=0
# am getting one match for 'Reza GHOOCHANNEJHAD'
# 316
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mehdi+taremi&x=0&y=0
# am getting one match for 'Mehdi TAREMI'
# 317
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alireza+jahanbakhsh&x=0&y=0
# am getting one match for 'Alireza JAHANBAKHSH'
# 318
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=majid+hosseini&x=0&y=0
# am getting one match for 'Majid HOSSEINI'
# 319
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sardar+azmoun&x=0&y=0
# am getting one match for 'Sardar AZMOUN'
# 320
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ashkan+dejagah&x=0&y=0
# am getting one match for 'Ashkan DEJAGAH'
# 321
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=amir+abedzadeh&x=0&y=0
# am getting one match for 'Amir ABEDZADEH'
# 322
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ramin+rezaeian&x=0&y=0
# am getting one match for 'Ramin REZAEIAN'
# 323
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=eiji+kawashima&x=0&y=0
# am getting one match for 'Eiji KAWASHIMA'
# 324
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=naomichi+ueda&x=0&y=0
# am getting one match for 'Naomichi UEDA'
# 325
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gen+shoji&x=0&y=0
# am getting one match for 'Gen SHOJI'
# 326
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=keisuke+honda&x=0&y=0
# am getting one match for 'Keisuke HONDA'
# 327
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yuto+nagatomo&x=0&y=0
# am getting one match for 'Yuto NAGATOMO'
# 328
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=wataru+endo&x=0&y=0
# am getting one match for 'Wataru ENDO'
# 329
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gaku+shibasaki&x=0&y=0
# am getting one match for 'Gaku SHIBASAKI'
# 330
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=genki+haraguchi&x=0&y=0
# am getting one match for 'Genki HARAGUCHI'
# 331
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=shinji+okazaki&x=0&y=0
# am getting one match for 'Shinji OKAZAKI'
# 332
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=shinji+kagawa&x=0&y=0
# am getting one match for 'Shinji KAGAWA'
# 333
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=takashi+usami&x=0&y=0
# am getting one match for 'Takashi USAMI'
# 334
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=masaaki+higashiguchi&x=0&y=0
# am getting one match for 'Masaaki HIGASHIGUCHI'
# 335
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yoshinori+muto&x=0&y=0
# am getting one match for 'Yoshinori MUTO'
# 336
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=takashi+inui&x=0&y=0
# am getting one match for 'Takashi INUI'
# 337
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yuya+osako&x=0&y=0
# am getting one match for 'Yuya OSAKO'
# 338
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hotaru+yamaguchi&x=0&y=0
# am getting one match for 'Hotaru YAMAGUCHI'
# 339
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=makoto+hasebe&x=0&y=0
# am getting one match for 'Makoto HASEBE'
# 340
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ryota+ohshima&x=0&y=0
# no results found for 'Ryota OHSHIMA'
# 341
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hiroki+sakai&x=0&y=0
# am getting one match for 'Hiroki SAKAI'
# 342
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tomoaki+makino&x=0&y=0
# am getting one match for 'Tomoaki MAKINO'
# 343
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gotoku+sakai&x=0&y=0
# am getting one match for 'Gotoku SAKAI'
# 344
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=maya+yoshida&x=0&y=0
# am getting one match for 'Maya YOSHIDA'
# 345
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kosuke+nakamura&x=0&y=0
# am getting one match for 'Kosuke NAKAMURA'
# 346
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kim+seunggyu&x=0&y=0
# no results found for 'KIM Seunggyu'
# 347
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lee+yong&x=0&y=0
# no results found for 'LEE Yong'
# 348
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jung+seunghyun&x=0&y=0
# no results found for 'JUNG Seunghyun'
# 349
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oh+bansuk&x=0&y=0
# no results found for 'OH Bansuk'
# 350
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yun+youngsun&x=0&y=0
# no results found for 'YUN Youngsun'
# 351
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=park+jooho&x=0&y=0
# no results found for 'PARK Jooho'
# 352
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=son+heungmin&x=0&y=0
# no results found for 'SON Heungmin'
# 353
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ju+sejong&x=0&y=0
# ---------------------------------------------------------------------------
# IndexError                                Traceback (most recent call last)
# <ipython-input-34-721d27c8c0f4> in <module>()
#      64     try:
# ---> 65         treex.xpath("//*[@id='main']/div[8]/div/div[1]/div[1]/h2/text()")[0]
#      66 

# IndexError: list index out of range

# During handling of the above exception, another exception occurred:

# IndexError                                Traceback (most recent call last)
# <ipython-input-34-721d27c8c0f4> in <module>()
#      91 #                     del row_player_url_deeper
#      92 
# ---> 93             row_player_age_string = row.xpath("td[@class='zentriert'][3]/text()")[0]
#      94 
#      95             try:

# IndexError: list index out of range

# 354 missing

# 355
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lee+seungwoo&x=0&y=0
# no results found for 'LEE Seungwoo'
# 356
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hwang+heechan&x=0&y=0
# no results found for 'HWANG Heechan'
# 357
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kim+minwoo&x=0&y=0
# no results found for 'KIM Minwoo'
# 358
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=koo+jacheol&x=0&y=0
# no results found for 'KOO Jacheol'
# 359
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hong+chul&x=0&y=0
# no results found for 'HONG Chul'
# 360
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jung+wooyoung&x=0&y=0
# no results found for 'JUNG Wooyoung'
# 361
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ki+sungyueng&x=0&y=0
# no results found for 'KI Sungyueng'
# 362
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lee+jaesung&x=0&y=0
# no results found for 'LEE Jaesung'
# 363
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=moon+seonmin&x=0&y=0
# no results found for 'MOON Seonmin'
# 364
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kim+younggwon&x=0&y=0
# no results found for 'KIM Younggwon'
# 365
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jang+hyunsoo&x=0&y=0
# no results found for 'JANG Hyunsoo'
# 366
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kim+jinhyeon&x=0&y=0
# no results found for 'KIM Jinhyeon'
# 367
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=go+yohan&x=0&y=0
# no results found for 'GO Yohan'
# 368
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jo+hyeonwoo&x=0&y=0
# no results found for 'JO Hyeonwoo'
# 369
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+corona&x=0&y=0
# am getting results, but no matches for 'Jose CORONA'
# 370
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hugo+ayala&x=0&y=0
# am getting one match for 'Hugo AYALA'
# 371
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+salcedo&x=0&y=0
# am getting one match for 'Carlos SALCEDO'
# 372
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rafael+marquez&x=0&y=0
# am getting one match for 'Rafael MARQUEZ'
# 373
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=diego+reyes&x=0&y=0
# am getting one match for 'Diego REYES'
# 374
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jonathan+dos+santos&x=0&y=0
# am getting one match for 'Jonathan DOS SANTOS'
# 375
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=miguel+layun&x=0&y=0
# am getting one match for 'Miguel LAYUN'
# 376
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marco+fabian&x=0&y=0
# am getting one match for 'Marco FABIAN'
# 377
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=raul+jimenez&x=0&y=0
# am getting one match for 'Raul JIMENEZ'
# 378
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=giovani+dos+santos&x=0&y=0
# am getting one match for 'Giovani DOS SANTOS'
# 379
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+vela&x=0&y=0
# am getting one match for 'Carlos VELA'
# 380
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alfredo+talavera&x=0&y=0
# am getting one match for 'Alfredo TALAVERA'
# 381
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=guillermo+ochoa&x=0&y=0
# am getting one match for 'Guillermo OCHOA'
# 382
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=javier+hernandez&x=0&y=0
# am getting results, but no matches for 'Javier HERNANDEZ'
# 383
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hector+moreno&x=0&y=0
# am getting one match for 'Hector MORENO'
# 384
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hector+herrera&x=0&y=0
# am getting one match for 'Hector HERRERA'
# 385
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jesus+corona&x=0&y=0
# am getting one match for 'Jesus CORONA'
# 386
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andres+guardado&x=0&y=0
# am getting one match for 'Andres GUARDADO'
# 387
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oribe+peralta&x=0&y=0
# am getting one match for 'Oribe PERALTA'
# 388
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=javier+aquino&x=0&y=0
# am getting one match for 'Javier AQUINO'
# 389
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=edson+alvarez&x=0&y=0
# am getting one match for 'Edson ALVAREZ'
# 390
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hirving+lozano&x=0&y=0
# am getting one match for 'Hirving LOZANO'
# 391
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jesus+gallardo&x=0&y=0
# am getting one match for 'Jesus GALLARDO'
# 392
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yassine+bounou&x=0&y=0
# am getting results, but no matches for 'Yassine BOUNOU'
# 393
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=achraf+hakimi&x=0&y=0
# am getting one match for 'Achraf HAKIMI'
# 394
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hamza+mendyl&x=0&y=0
# am getting one match for 'Hamza MENDYL'
# 395
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=manuel+da+costa&x=0&y=0
# am getting one match for 'Manuel DA COSTA'
# 396
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mehdi+benatia&x=0&y=0
# am getting results, but no matches for 'Mehdi BENATIA'
# 397
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=romain+saiss&x=0&y=0
# am getting one match for 'Romain SAISS'
# 398
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hakim+ziyach&x=0&y=0
# no results found for 'Hakim ZIYACH'
# 399
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=karim+el+ahmadi&x=0&y=0
# am getting one match for 'Karim EL AHMADI'
# 400
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ayoub+el+kaabi&x=0&y=0
# am getting one match for 'Ayoub EL KAABI'
# 401
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=younes+belhanda&x=0&y=0
# am getting one match for 'Younes BELHANDA'
# 402
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=faycal+fajr&x=0&y=0
# am getting one match for 'Faycal FAJR'
# 403
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=monir+el+kajoui&x=0&y=0
# no results found for 'Monir EL KAJOUI'
# 404
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=khalid+boutaib&x=0&y=0
# am getting one match for 'Khalid BOUTAIB'
# 405
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mbark+boussoufa&x=0&y=0
# no results found for 'Mbark BOUSSOUFA'
# 406
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=youssef+ait+bennasser&x=0&y=0
# am getting one match for 'Youssef AIT BENNASSER'
# 407
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=noureddine+amrabat&x=0&y=0
# no results found for 'Noureddine AMRABAT'
# 408
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nabil+dirar&x=0&y=0
# am getting one match for 'Nabil DIRAR'
# 409
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=amine+harit&x=0&y=0
# am getting one match for 'Amine HARIT'
# 410
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=youssef+en+nesyri&x=0&y=0
# no results found for 'Youssef EN NESYRI'
# 411
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aziz+bouhaddouz&x=0&y=0
# am getting one match for 'Aziz BOUHADDOUZ'
# 412
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sofyan+amrabat&x=0&y=0
# am getting one match for 'Sofyan AMRABAT'
# 413
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ahmed+tagnaouti&x=0&y=0
# no results found for 'Ahmed TAGNAOUTI'
# 414
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mehdi+carcela&x=0&y=0
# am getting one match for 'Mehdi CARCELA'
# 415
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ikechukwu+ezenwa&x=0&y=0
# am getting one match for 'Ikechukwu EZENWA'
# 416
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bryan+idowu&x=0&y=0
# am getting one match for 'Bryan IDOWU'
# 417
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=elderson+echiejile&x=0&y=0
# am getting results, but no matches for 'Elderson ECHIEJILE'
# 418
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=onyinye+ndidi&x=0&y=0
# no results found for 'Onyinye NDIDI'
# 419
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=william+ekong&x=0&y=0
# no results found for 'William EKONG'
# 420
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=leon+balogun&x=0&y=0
# am getting one match for 'Leon BALOGUN'
# 421
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ahmed+musa&x=0&y=0
# am getting one match for 'Ahmed MUSA'
# 422
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oghenekaro+etebo&x=0&y=0
# am getting one match for 'Oghenekaro ETEBO'
# 423
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=odion+ighalo&x=0&y=0
# am getting one match for 'Odion IGHALO'
# 424
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=john+obi+mikel&x=0&y=0
# no results found for 'John Obi MIKEL'
# 425
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=victor+moses&x=0&y=0
# am getting one match for 'Victor MOSES'
# 426
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdullahi+shehu&x=0&y=0
# am getting one match for 'Abdullahi SHEHU'
# 427
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=simeon+nwankwo&x=0&y=0
# no results found for 'Simeon NWANKWO'
# 428
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kelechi+iheanacho&x=0&y=0
# am getting one match for 'Kelechi IHEANACHO'
# 429
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=joel+obi&x=0&y=0
# am getting one match for 'Joel OBI'
# 430
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=daniel+akpeyi&x=0&y=0
# am getting one match for 'Daniel AKPEYI'
# 431
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ogenyi+onazi&x=0&y=0
# am getting one match for 'Ogenyi ONAZI'
# 432
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alex+iwobi&x=0&y=0
# am getting one match for 'Alex IWOBI'
# 433
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=john+ogu&x=0&y=0
# am getting one match for 'John OGU'
# 434
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=chidozie+awaziem&x=0&y=0
# am getting one match for 'Chidozie AWAZIEM'
# 435
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=tyronne+ebuehi&x=0&y=0
# am getting one match for 'Tyronne EBUEHI'
# 436
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kenneth+omeruo&x=0&y=0
# am getting one match for 'Kenneth OMERUO'
# 437
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=francis+uzoho&x=0&y=0
# am getting one match for 'Francis UZOHO'
# 438
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jaime+penedo&x=0&y=0
# am getting one match for 'Jaime PENEDO'
# 439
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=michael+murillo&x=0&y=0
# am getting one match for 'Michael MURILLO'
# 440
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=harold+cummings&x=0&y=0
# am getting one match for 'Harold CUMMINGS'
# 441
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fidel+escobar&x=0&y=0
# am getting one match for 'Fidel ESCOBAR'
# 442
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=roman+torres&x=0&y=0
# am getting one match for 'Roman TORRES'
# 443
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gabriel+gomez&x=0&y=0
# am getting one match for 'Gabriel GOMEZ'
# 444
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=blas+perez&x=0&y=0
# am getting one match for 'Blas PEREZ'
# 445
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=edgar+barcenas&x=0&y=0
# no results found for 'Edgar BARCENAS'
# 446
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gabriel+torres&x=0&y=0
# am getting one match for 'Gabriel TORRES'
# 447
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ismael+diaz&x=0&y=0
# am getting one match for 'Ismael DIAZ'
# 448
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=armando+cooper&x=0&y=0
# am getting one match for 'Armando COOPER'
# 449
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+calderon&x=0&y=0
# am getting one match for 'Jose CALDERON'
# 450
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=adolfo+machado&x=0&y=0
# am getting one match for 'Adolfo MACHADO'
# 451
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=valentin+pimentel&x=0&y=0
# am getting one match for 'Valentin PIMENTEL'
# 452
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=eric+davis&x=0&y=0
# am getting one match for 'Eric DAVIS'
# 453
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdiel+arroyo&x=0&y=0
# am getting one match for 'Abdiel ARROYO'
# 454
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luis+ovalle&x=0&y=0
# am getting one match for 'Luis OVALLE'
# 455
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luis+tejada&x=0&y=0
# am getting one match for 'Luis TEJADA'
# 456
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alberto+quintero&x=0&y=0
# am getting one match for 'Alberto QUINTERO'
# 457
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=anibal+godoy&x=0&y=0
# am getting one match for 'Anibal GODOY'
# 458
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+luis+rodriguez&x=0&y=0
# am getting one match for 'Jose Luis RODRIGUEZ'
# 459
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alex+rodriguez&x=0&y=0
# am getting one match for 'Alex RODRIGUEZ'
# 460
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=felipe+baloy&x=0&y=0
# am getting one match for 'Felipe BALOY'
# 461
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pedro+gallese&x=0&y=0
# am getting one match for 'Pedro GALLESE'
# 462
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alberto+rodriguez&x=0&y=0
# am getting one match for 'Alberto RODRIGUEZ'
# 463
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aldo+corzo&x=0&y=0
# am getting one match for 'Aldo CORZO'
# 464
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=anderson+santamaria&x=0&y=0
# am getting one match for 'Anderson SANTAMARIA'
# 465
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=miguel+araujo&x=0&y=0
# am getting one match for 'Miguel ARAUJO'
# 466
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=miguel+trauco&x=0&y=0
# am getting one match for 'Miguel TRAUCO'
# 467
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=paolo+hurtado&x=0&y=0
# am getting one match for 'Paolo HURTADO'
# 468
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=christian+cueva&x=0&y=0
# am getting one match for 'Christian CUEVA'
# 469
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=paolo+guerrero&x=0&y=0
# am getting one match for 'Paolo GUERRERO'
# 470
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jefferson+farfan&x=0&y=0
# am getting one match for 'Jefferson FARFAN'
# 471
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=raul+ruidiaz&x=0&y=0
# am getting one match for 'Raul RUIDIAZ'
# 472
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+caceda&x=0&y=0
# am getting one match for 'Carlos CACEDA'
# 473
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=renato+tapia&x=0&y=0
# am getting one match for 'Renato TAPIA'
# 474
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andy+polo&x=0&y=0
# am getting one match for 'Andy POLO'
# 475
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=christian+ramos&x=0&y=0
# am getting one match for 'Christian RAMOS'
# 476
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=wilder+cartagena&x=0&y=0
# am getting one match for 'Wilder CARTAGENA'
# 477
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luis+advincula&x=0&y=0
# am getting one match for 'Luis ADVINCULA'
# 478
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andre+carrillo&x=0&y=0
# am getting one match for 'Andre CARRILLO'
# 479
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yoshimar+yotun&x=0&y=0
# am getting one match for 'Yoshimar YOTUN'
# 480
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=edison+flores&x=0&y=0
# am getting one match for 'Edison FLORES'
# 481
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+carvallo&x=0&y=0
# am getting one match for 'Jose CARVALLO'
# 482
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nilson+loyola&x=0&y=0
# am getting one match for 'Nilson LOYOLA'
# 483
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pedro+aquino&x=0&y=0
# am getting one match for 'Pedro AQUINO'
# 484
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=wojciech+szczesny&x=0&y=0
# am getting one match for 'Wojciech SZCZESNY'
# 485
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=michal+pazdan&x=0&y=0
# am getting one match for 'Michal PAZDAN'
# 486
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=artur+jedrzejczyk&x=0&y=0
# am getting one match for 'Artur JEDRZEJCZYK'
# 487
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thiago+cionek&x=0&y=0
# am getting one match for 'Thiago CIONEK'
# 488
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jan+bednarek&x=0&y=0
# am getting one match for 'Jan BEDNAREK'
# 489
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jacek+goralski&x=0&y=0
# am getting one match for 'Jacek GORALSKI'
# 490
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=arkadiusz+milik&x=0&y=0
# am getting one match for 'Arkadiusz MILIK'
# 491
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=karol+linetty&x=0&y=0
# am getting one match for 'Karol LINETTY'
# 492
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=robert+lewandowski&x=0&y=0
# am getting one match for 'Robert LEWANDOWSKI'
# 493
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=grzegorz+krychowiak&x=0&y=0
# am getting one match for 'Grzegorz KRYCHOWIAK'
# 494
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kamil+grosicki&x=0&y=0
# am getting one match for 'Kamil GROSICKI'
# 495
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bartosz+bialkowski&x=0&y=0
# am getting one match for 'Bartosz BIALKOWSKI'
# 496
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=maciej+rybus&x=0&y=0
# am getting one match for 'Maciej RYBUS'
# 497
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lukasz+teodorczyk&x=0&y=0
# am getting one match for 'Lukasz TEODORCZYK'
# 498
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kamil+glik&x=0&y=0
# am getting one match for 'Kamil GLIK'
# 499
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jakub+blaszczykowski&x=0&y=0
# am getting one match for 'Jakub BLASZCZYKOWSKI'
# 500
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=slawomir+peszko&x=0&y=0
# am getting one match for 'Slawomir PESZKO'
# 501
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bartosz+bereszynski&x=0&y=0
# am getting one match for 'Bartosz BERESZYNSKI'
# 502
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=piotr+zielinski&x=0&y=0
# am getting one match for 'Piotr ZIELINSKI'
# 503
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lukasz+piszczek&x=0&y=0
# am getting one match for 'Lukasz PISZCZEK'
# 504
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rafal+kurzawa&x=0&y=0
# am getting one match for 'Rafal KURZAWA'
# 505
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lukasz+fabianski&x=0&y=0
# am getting one match for 'Lukasz FABIANSKI'
# 506
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dawid+kownacki&x=0&y=0
# am getting one match for 'Dawid KOWNACKI'
# 507
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rui+patricio&x=0&y=0
# am getting one match for 'RUI PATRICIO'
# 508
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bruno+alves&x=0&y=0
# am getting one match for 'BRUNO ALVES'
# 509
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pepe&x=0&y=0
# am getting one match for 'PEPE'
# 510
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=manuel+fernandes&x=0&y=0
# am getting one match for 'MANUEL FERNANDES'
# 511
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=raphael+guerreiro&x=0&y=0
# am getting one match for 'RAPHAEL GUERREIRO'
# 512
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+fonte&x=0&y=0
# am getting one match for 'JOSE FONTE'
# 513
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristiano+ronaldo&x=0&y=0
# am getting one match for 'CRISTIANO RONALDO'
# 514
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=joao+moutinho&x=0&y=0
# am getting one match for 'JOAO MOUTINHO'
# 515
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andre+silva&x=0&y=0
# am getting one match for 'ANDRE SILVA'
# 516
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=joao+mario&x=0&y=0
# am getting one match for 'JOAO MARIO'
# 517
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bernardo+silva&x=0&y=0
# am getting one match for 'BERNARDO SILVA'
# 518
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=anthony+lopes&x=0&y=0
# am getting one match for 'ANTHONY LOPES'
# 519
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ruben+dias&x=0&y=0
# am getting one match for 'RUBEN DIAS'
# 520
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=william&x=0&y=0
# am getting one match for 'WILLIAM'
# 521
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ricardo&x=0&y=0
# am getting multiple matches for 'RICARDO'
# 522
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bruno+fernandes&x=0&y=0
# am getting one match for 'BRUNO FERNANDES'
# 523
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=goncalo+guedes&x=0&y=0
# am getting one match for 'GONCALO GUEDES'
# 524
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gelson+martins&x=0&y=0
# am getting one match for 'GELSON MARTINS'
# 525
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mario+rui&x=0&y=0
# am getting one match for 'MARIO RUI'
# 526
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ricardo+quaresma&x=0&y=0
# am getting one match for 'RICARDO QUARESMA'
# 527
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cedric&x=0&y=0
# am getting one match for 'CEDRIC'
# 528
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=beto&x=0&y=0
# am getting one match for 'BETO'
# 529
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=adrien+silva&x=0&y=0
# am getting one match for 'ADRIEN SILVA'
# 530
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=igor+akinfeev&x=0&y=0
# am getting one match for 'Igor AKINFEEV'
# 531
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mario+fernandes&x=0&y=0
# am getting one match for 'MARIO FERNANDES'
# 532
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ilya+kutepov&x=0&y=0
# am getting one match for 'Ilya KUTEPOV'
# 533
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sergey+ignashevich&x=0&y=0
# am getting one match for 'Sergey IGNASHEVICH'
# 534
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andrey+semenov&x=0&y=0
# am getting one match for 'Andrey SEMENOV'
# 535
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=denis+cheryshev&x=0&y=0
# am getting one match for 'Denis CHERYSHEV'
# 536
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=daler+kuziaev&x=0&y=0
# no results found for 'Daler KUZIAEV'
# 537
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=iury+gazinsky&x=0&y=0
# no results found for 'Iury GAZINSKY'
# 538
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alan+dzagoev&x=0&y=0
# am getting one match for 'Alan DZAGOEV'
# 539
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fedor+smolov&x=0&y=0
# am getting one match for 'Fedor SMOLOV'
# 540
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=roman+zobnin&x=0&y=0
# am getting one match for 'Roman ZOBNIN'
# 541
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andrei+lunev&x=0&y=0
# no results found for 'Andrei LUNEV'
# 542
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fedor+kudriashov&x=0&y=0
# no results found for 'Fedor KUDRIASHOV'
# 543
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=vladimir+granat&x=0&y=0
# am getting one match for 'Vladimir GRANAT'
# 544
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alexey+miranchuk&x=0&y=0
# no results found for 'Alexey MIRANCHUK'
# 545
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=anton+miranchuk&x=0&y=0
# am getting one match for 'Anton MIRANCHUK'
# 546
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aleksandr+golovin&x=0&y=0
# am getting one match for 'Aleksandr GOLOVIN'
# 547
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yury+zhirkov&x=0&y=0
# no results found for 'Yury ZHIRKOV'
# 548
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alexander+samedov&x=0&y=0
# no results found for 'Alexander SAMEDOV'
# 549
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=vladimir+gabulov&x=0&y=0
# am getting one match for 'Vladimir GABULOV'
# 550
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aleksandr+erokhin&x=0&y=0
# am getting one match for 'Aleksandr EROKHIN'
# 551
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=artem+dzyuba&x=0&y=0
# am getting one match for 'Artem DZYUBA'
# 552
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=igor+smolnikov&x=0&y=0
# am getting one match for 'Igor SMOLNIKOV'
# 553
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdullah+almuaiouf&x=0&y=0
# no results found for 'ABDULLAH ALMUAIOUF'
# 554
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mansour+alharbi&x=0&y=0
# no results found for 'MANSOUR ALHARBI'
# 555
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=osama+hawsawi&x=0&y=0
# am getting one match for 'OSAMA HAWSAWI'
# 556
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ali+albulayhi&x=0&y=0
# no results found for 'ALI ALBULAYHI'
# 557
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=omar+othman&x=0&y=0
# am getting results, but no matches for 'OMAR OTHMAN'
# 558
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohammed+alburayk&x=0&y=0
# no results found for 'MOHAMMED ALBURAYK'
# 559
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=salman+alfaraj&x=0&y=0
# no results found for 'SALMAN ALFARAJ'
# 560
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yahia+alshehri&x=0&y=0
# no results found for 'YAHIA ALSHEHRI'
# 561
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hatan+bahbir&x=0&y=0
# no results found for 'HATAN BAHBIR'
# 562
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohammed+alsahlawi&x=0&y=0
# no results found for 'MOHAMMED ALSAHLAWI'
# 563
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdulmalek+alkhaibri&x=0&y=0
# no results found for 'ABDULMALEK ALKHAIBRI'
# 564
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohamed+kanno&x=0&y=0
# am getting one match for 'MOHAMED KANNO'
# 565
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yasser+alshahrani&x=0&y=0
# no results found for 'YASSER ALSHAHRANI'
# 566
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdullah+otayf&x=0&y=0
# no results found for 'ABDULLAH OTAYF'
# 567
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdullah+alkhaibari&x=0&y=0
# no results found for 'ABDULLAH ALKHAIBARI'
# 568
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hussain+almoqahwi&x=0&y=0
# no results found for 'HUSSAIN ALMOQAHWI'
# 569
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=taiseer+aljassam&x=0&y=0
# no results found for 'TAISEER ALJASSAM'
# 570
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=salem+aldawsari&x=0&y=0
# no results found for 'SALEM ALDAWSARI'
# 571
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fahad+almuwallad&x=0&y=0
# no results found for 'FAHAD ALMUWALLAD'
# 572
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=muhannad+asiri&x=0&y=0
# am getting one match for 'MUHANNAD ASIRI'
# 573
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yasser+almosailem&x=0&y=0
# no results found for 'YASSER ALMOSAILEM'
# 574
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohammed+alowais&x=0&y=0
# no results found for 'MOHAMMED ALOWAIS'
# 575
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=motaz+hawsawi&x=0&y=0
# am getting one match for 'MOTAZ HAWSAWI'
# 576
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=abdoulaye+diallo&x=0&y=0
# am getting multiple matches for 'Abdoulaye DIALLO'
# 577
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=saliou+ciss&x=0&y=0
# am getting one match for 'Saliou CISS'
# 578
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kalidou+koulibaly&x=0&y=0
# am getting one match for 'Kalidou KOULIBALY'
# 579
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kara+mbodji&x=0&y=0
# no results found for 'Kara MBODJI'
# 580
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=idrissa+gana+gueye&x=0&y=0
# am getting results, but no matches for 'Idrissa Gana GUEYE'
# 581
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=salif+sane&x=0&y=0
# am getting one match for 'Salif SANE'
# 582
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=moussa+sow&x=0&y=0
# am getting one match for 'Moussa SOW'
# 583
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cheikhou+kouyate&x=0&y=0
# am getting one match for 'Cheikhou KOUYATE'
# 584
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mame+diouf&x=0&y=0
# am getting one match for 'Mame DIOUF'
# 585
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sadio+mane&x=0&y=0
# am getting one match for 'Sadio MANE'
# 586
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cheikh+ndoye&x=0&y=0
# am getting one match for 'Cheikh NDOYE'
# 587
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=youssouf+sabaly&x=0&y=0
# am getting one match for 'Youssouf SABALY'
# 588
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alfred+ndiaye&x=0&y=0
# no results found for 'Alfred NDIAYE'
# 589
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=moussa+konate&x=0&y=0
# am getting one match for 'Moussa KONATE'
# 590
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=diafra+sakho&x=0&y=0
# am getting one match for 'Diafra SAKHO'
# 591
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=khadim+ndiaye&x=0&y=0
# no results found for 'Khadim NDIAYE'
# 592
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pape+alioune+ndiaye&x=0&y=0
# no results found for 'Pape Alioune NDIAYE'
# 593
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ismaila+sarr&x=0&y=0
# am getting one match for 'Ismaila SARR'
# 594
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mbaye+niang&x=0&y=0
# am getting one match for 'Mbaye NIANG'
# 595
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=keita+balde&x=0&y=0
# am getting one match for 'Keita BALDE'
# 596
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lamine+gassama&x=0&y=0
# am getting one match for 'Lamine GASSAMA'
# 597
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=moussa+wague&x=0&y=0
# am getting one match for 'Moussa WAGUE'
# 598
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alfred+gomis&x=0&y=0
# am getting one match for 'Alfred GOMIS'
# 599
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=vladimir+stojkovic&x=0&y=0
# am getting one match for 'Vladimir STOJKOVIC'
# 600
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=antonio+rukavina&x=0&y=0
# am getting one match for 'Antonio RUKAVINA'
# 601
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dusko+tosic&x=0&y=0
# am getting one match for 'Dusko TOSIC'
# 602
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luka+milivojevic&x=0&y=0
# am getting one match for 'Luka MILIVOJEVIC'
# 603
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=uros+spajic&x=0&y=0
# am getting one match for 'Uros SPAJIC'
# 604
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=branislav+ivanovic&x=0&y=0
# am getting one match for 'Branislav IVANOVIC'
# 605
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andrija+zivkovic&x=0&y=0
# am getting one match for 'Andrija ZIVKOVIC'
# 606
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aleksandar+prijovic&x=0&y=0
# am getting one match for 'Aleksandar PRIJOVIC'
# 607
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aleksandar+mitrovic&x=0&y=0
# am getting one match for 'Aleksandar MITROVIC'
# 608
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dusan+tadic&x=0&y=0
# am getting one match for 'Dusan TADIC'
# 609
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aleksandar+kolarov&x=0&y=0
# am getting one match for 'Aleksandar KOLAROV'
# 610
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=predrag+rajkovic&x=0&y=0
# am getting one match for 'Predrag RAJKOVIC'
# 611
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=milos+veljkovic&x=0&y=0
# am getting one match for 'Milos VELJKOVIC'
# 612
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=milan+rodic&x=0&y=0
# am getting one match for 'Milan RODIC'
# 613
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nikola+milenkovic&x=0&y=0
# am getting one match for 'Nikola MILENKOVIC'
# 614
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marko+grujic&x=0&y=0
# am getting one match for 'Marko GRUJIC'
# 615
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=filip+kostic&x=0&y=0
# am getting multiple matches for 'Filip KOSTIC'
# 616
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nemanja+radonjic&x=0&y=0
# am getting one match for 'Nemanja RADONJIC'
# 617
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luka+jovic&x=0&y=0
# am getting one match for 'Luka JOVIC'
# 618
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sergej+milinkovic-savic&x=0&y=0
# am getting results, but no matches for 'Sergej MILINKOVIC-SAVIC'
# 619
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nemanja+matic&x=0&y=0
# am getting one match for 'Nemanja MATIC'
# 620
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=adem+ljajic&x=0&y=0
# am getting one match for 'Adem LJAJIC'
# 621
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marko+dmitrovic&x=0&y=0
# am getting one match for 'Marko DMITROVIC'
# 622
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=david+de+gea&x=0&y=0
# am getting one match for 'David DE GEA'
# 623
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dani+carvajal&x=0&y=0
# no results found for 'Dani CARVAJAL'
# 624
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gerard+pique&x=0&y=0
# am getting one match for 'Gerard PIQUE'
# 625
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nacho&x=0&y=0
# am getting multiple matches for 'NACHO'
# 626
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sergio+busquets&x=0&y=0
# am getting one match for 'Sergio BUSQUETS'
# 627
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andres+iniesta&x=0&y=0
# am getting one match for 'Andres INIESTA'
# 628
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=saul&x=0&y=0
# am getting multiple matches for 'SAUL'
# 629
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=koke&x=0&y=0
# am getting one match for 'KOKE'
# 630
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rodrigo&x=0&y=0
# am getting one match for 'RODRIGO'
# 631
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=thiago&x=0&y=0
# am getting one match for 'THIAGO'
# 632
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lucas+vazquez&x=0&y=0
# am getting one match for 'Lucas VAZQUEZ'
# 633
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=alvaro+odriozola&x=0&y=0
# am getting one match for 'Alvaro ODRIOZOLA'
# 634
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kepa+arrizabalaga&x=0&y=0
# am getting one match for 'Kepa ARRIZABALAGA'
# 635
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cesar+azpilicueta&x=0&y=0
# am getting one match for 'Cesar AZPILICUETA'
# 636
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sergio+ramos&x=0&y=0
# am getting one match for 'Sergio RAMOS'
# 637
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nacho+monreal&x=0&y=0
# am getting one match for 'Nacho MONREAL'
# 638
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=iago+aspas&x=0&y=0
# am getting one match for 'Iago ASPAS'
# 639
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jordi+alba&x=0&y=0
# am getting one match for 'Jordi ALBA'
# 640
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=diego+costa&x=0&y=0
# am getting one match for 'Diego COSTA'
# 641
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marco+asensio&x=0&y=0
# am getting one match for 'Marco ASENSIO'
# 642
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=david+silva&x=0&y=0
# am getting one match for 'David SILVA'
# 643
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=isco&x=0&y=0
# am getting one match for 'ISCO'
# 644
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pepe+reina&x=0&y=0
# am getting one match for 'Pepe REINA'
# 645
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=robin+olsen&x=0&y=0
# am getting one match for 'Robin OLSEN'
# 646
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mikael+lustig&x=0&y=0
# am getting one match for 'Mikael LUSTIG'
# 647
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=victor+lindelof&x=0&y=0
# am getting one match for 'Victor LINDELOF'
# 648
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=andreas+granqvist&x=0&y=0
# am getting one match for 'Andreas GRANQVIST'
# 649
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=martin+olsson&x=0&y=0
# am getting one match for 'Martin OLSSON'
# 650
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ludwig+augustinsson&x=0&y=0
# am getting one match for 'Ludwig AUGUSTINSSON'
# 651
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sebastian+larsson&x=0&y=0
# am getting one match for 'Sebastian LARSSON'
# 652
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=albin+ekdal&x=0&y=0
# am getting one match for 'Albin EKDAL'
# 653
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcus+berg&x=0&y=0
# am getting one match for 'Marcus BERG'
# 654
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=emil+forsberg&x=0&y=0
# am getting one match for 'Emil FORSBERG'
# 655
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=john+guidetti&x=0&y=0
# am getting one match for 'John GUIDETTI'

# >>>>>>>>>>>>XXXXXXXXXXXXXXXXXXX

# 656
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=karl-johan+johnsson&x=0&y=0
# am getting results, but no matches for 'Karl-Johan JOHNSSON'
# 657
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gustav+svensson&x=0&y=0
# am getting one match for 'Gustav SVENSSON'
# 658
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=filip+helander&x=0&y=0
# am getting one match for 'Filip HELANDER'
# 659
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oscar+hiljemark&x=0&y=0
# am getting one match for 'Oscar HILJEMARK'
# 660
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=emil+krafth&x=0&y=0
# am getting one match for 'Emil KRAFTH'
# 661
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=viktor+claesson&x=0&y=0
# am getting one match for 'Viktor CLAESSON'
# 662
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=pontus+jansson&x=0&y=0
# am getting one match for 'Pontus JANSSON'
# 663
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=marcus+rohden&x=0&y=0
# am getting one match for 'Marcus ROHDEN'
# 664
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ola+toivonen&x=0&y=0
# am getting one match for 'Ola TOIVONEN'
# 665
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jimmy+durmaz&x=0&y=0
# am getting one match for 'Jimmy DURMAZ'
# 666
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=isaac+kiese+thelin&x=0&y=0
# am getting results, but no matches for 'Isaac Kiese THELIN'
# 667
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=kristoffer+nordfeldt&x=0&y=0
# am getting one match for 'Kristoffer NORDFELDT'
# 668
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yann+sommer&x=0&y=0
# am getting one match for 'Yann SOMMER'
# 669
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=stephan+lichtsteiner&x=0&y=0
# am getting one match for 'Stephan LICHTSTEINER'
# 670
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=francois+moubandje&x=0&y=0
# am getting one match for 'Francois MOUBANDJE'
# 671
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nico+elvedi&x=0&y=0
# am getting one match for 'Nico ELVEDI'
# 672
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=manuel+akanji&x=0&y=0
# am getting one match for 'Manuel AKANJI'
# 673
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=michael+lang&x=0&y=0
# am getting one match for 'Michael LANG'
# 674
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=breel+embolo&x=0&y=0
# am getting one match for 'Breel EMBOLO'
# 675
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=remo+freuler&x=0&y=0
# am getting one match for 'Remo FREULER'
# 676
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=haris+seferovic&x=0&y=0
# am getting one match for 'Haris SEFEROVIC'
# 677
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=granit+xhaka&x=0&y=0
# am getting one match for 'Granit XHAKA'
# 678
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=valon+behrami&x=0&y=0
# am getting one match for 'Valon BEHRAMI'
# 679
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yvon+mvogo&x=0&y=0
# am getting one match for 'Yvon MVOGO'
# 680
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ricardo+rodriguez&x=0&y=0
# am getting one match for 'Ricardo RODRIGUEZ'
# 681
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=steven+zuber&x=0&y=0
# am getting one match for 'Steven ZUBER'
# 682
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=blerim+dzemaili&x=0&y=0
# am getting one match for 'Blerim DZEMAILI'
# 683
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gelson+fernandes&x=0&y=0
# am getting one match for 'Gelson FERNANDES'
# 684
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=denis+zakaria&x=0&y=0
# am getting one match for 'Denis ZAKARIA'
# 685
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mario+gavranovic&x=0&y=0
# am getting one match for 'Mario GAVRANOVIC'
# 686
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=josip+drmic&x=0&y=0
# am getting one match for 'Josip DRMIC'
# 687
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=johan+djourou&x=0&y=0
# am getting one match for 'Johan DJOUROU'
# 688
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=roman+buerki&x=0&y=0
# no results found for 'Roman BUERKI'
# 689
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fabian+schaer&x=0&y=0
# no results found for 'Fabian SCHAER'
# 690
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=xherdan+shaqiri&x=0&y=0
# am getting one match for 'Xherdan SHAQIRI'
# 691
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=farouk+ben+mustapha&x=0&y=0
# am getting one match for 'Farouk BEN MUSTAPHA'
# 692
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=syam+ben+youssef&x=0&y=0
# am getting one match for 'Syam BEN YOUSSEF'
# 693
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yohan+ben+alouane&x=0&y=0
# no results found for 'Yohan BEN ALOUANE'
# 694
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=yassine+meriah&x=0&y=0
# am getting one match for 'Yassine MERIAH'
# 695
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=oussama+haddadi&x=0&y=0
# am getting one match for 'Oussama HADDADI'
# 696
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rami+bedoui&x=0&y=0
# am getting one match for 'Rami BEDOUI'
# 697
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=saifeddine+khaoui&x=0&y=0
# no results found for 'Saifeddine KHAOUI'
# 698
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fakhreddine+ben+youssef&x=0&y=0
# am getting one match for 'Fakhreddine BEN YOUSSEF'
# 699
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=anice+badri&x=0&y=0
# am getting one match for 'Anice BADRI'
# 700
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=wahbi+khazri&x=0&y=0
# am getting one match for 'Wahbi KHAZRI'
# 701
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=dylan+bronn&x=0&y=0
# am getting one match for 'Dylan BRONN'
# 702
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ali+maaloul&x=0&y=0
# am getting one match for 'Ali MAALOUL'
# 703
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ferjani+sassi&x=0&y=0
# am getting one match for 'Ferjani SASSI'
# 704
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mohamed+ben+amor&x=0&y=0
# no results found for 'Mohamed BEN AMOR'
# 705
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ahmed+khalil&x=0&y=0
# am getting one match for 'Ahmed KHALIL'
# 706
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=aymen+mathlouthi&x=0&y=0
# am getting results, but no matches for 'Aymen MATHLOUTHI'
# 707
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ellyes+skhiri&x=0&y=0
# am getting one match for 'Ellyes SKHIRI'
# 708
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=bassem+srarfi&x=0&y=0
# am getting one match for 'Bassem SRARFI'
# 709
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=saber+khalifa&x=0&y=0
# am getting one match for 'Saber KHALIFA'
# 710
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=ghaylen+chaaleli&x=0&y=0
# no results found for 'Ghaylen CHAALELI'
# 711
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=hamdi+naguez&x=0&y=0
# no results found for 'Hamdi NAGUEZ'
# 712
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=mouez+hassen&x=0&y=0
# am getting one match for 'Mouez HASSEN'
# 713
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=naim+sliti&x=0&y=0
# am getting one match for 'Naim SLITI'
# 714
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=fernando+muslera&x=0&y=0
# am getting one match for 'Fernando MUSLERA'
# 715
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jose+gimenez&x=0&y=0
# am getting one match for 'Jose GIMENEZ'
# 716
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=diego+godin&x=0&y=0
# am getting one match for 'Diego GODIN'
# 717
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=guillermo+varela&x=0&y=0
# am getting one match for 'Guillermo VARELA'
# 718
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=carlos+sanchez&x=0&y=0
# am getting one match for 'Carlos SANCHEZ'
# 719
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=rodrigo+bentancur&x=0&y=0
# am getting one match for 'Rodrigo BENTANCUR'
# 720
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristian+rodriguez&x=0&y=0
# am getting one match for 'Cristian RODRIGUEZ'
# 721
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=nahitan+nandez&x=0&y=0
# am getting one match for 'Nahitan NANDEZ'
# 722
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=luis+suarez&x=0&y=0
# am getting one match for 'Luis SUAREZ'
# 723
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=giorgian+de+arrascaeta&x=0&y=0
# am getting one match for 'Giorgian DE ARRASCAETA'
# 724
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=cristhian+stuani&x=0&y=0
# am getting one match for 'Cristhian STUANI'
# 725
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=martin+campana&x=0&y=0
# am getting one match for 'Martin CAMPANA'
# 726
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=gaston+silva&x=0&y=0
# am getting one match for 'Gaston SILVA'
# 727
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=lucas+torreira&x=0&y=0
# am getting one match for 'Lucas TORREIRA'
# 728
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=matias+vecino&x=0&y=0
# am getting one match for 'Matias VECINO'
# 729
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=maximiliano+pereira&x=0&y=0
# am getting results, but no matches for 'Maximiliano PEREIRA'
# 730
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=diego+laxalt&x=0&y=0
# am getting one match for 'Diego LAXALT'
# 731
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=maximiliano+gomez&x=0&y=0
# am getting one match for 'Maximiliano GOMEZ'
# 732
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=sebastian+coates&x=0&y=0
# am getting one match for 'Sebastian COATES'
# 733
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=jonathan+urretaviscaya&x=0&y=0
# am getting one match for 'Jonathan URRETAVISCAYA'
# 734
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=edinson+cavani&x=0&y=0
# am getting one match for 'Edinson CAVANI'
# 735
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=martin+caceres&x=0&y=0
# am getting one match for 'Martin CACERES'
# 736
# https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=martin+silva&x=0&y=0
# am getting one match for 'Martin SILVA'
# done and done



In [40]:
#from here see which players don't give us urls

#put some check to make sure the page is downloaded before you move on to the next page
#like putting in the delay after making the request

headersx = {
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,en-GB;q=0.8",
"cache-control": "no-cache",
"cookie": "TMSESSID=49unp8sbp99moc95uhqljv0ob4; utag_main=v_id:0163d8d03800001c6172f1bd893304069002006100868$_sn:3$_ss:0$_st:1528450968811$dc_visit_transfermarkt-transfermarkt.de:3$ses_id:1528446062028%3Bexp-session$_pn:18%3Bexp-session$dc_event_transfermarkt-transfermarkt.de:18%3Bexp-session",
"pragma": "no-cache",
"referer": "https://www.transfermarkt.co.uk",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
}

url_store_1 = pd.read_csv('data/player_url_store_combined_part_1.csv')

player_df = pd.read_csv('data/fifa_player_list.csv')

index_list_available = list(url_store_1['index_number'].unique())

index_list_total = list(player_df['index'].unique())

not_there_list = [x for x in index_list_total if x not in index_list_available]

print(not_there_list)

[23, 44, 65, 70, 71, 81, 84, 85, 86, 88, 91, 101, 117, 135, 146, 153, 163, 165, 174, 181, 185, 190, 191, 194, 197, 198, 203, 204, 207, 228, 229, 243, 263, 265, 266, 268, 269, 274, 275, 277, 278, 281, 283, 289, 291, 292, 297, 298, 299, 300, 302, 304, 340, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 382, 392, 396, 398, 403, 405, 407, 410, 413, 417, 418, 419, 424, 427, 445, 521, 536, 537, 541, 542, 544, 547, 548, 553, 554, 556, 557, 558, 559, 560, 561, 562, 563, 565, 566, 567, 568, 569, 570, 571, 573, 574, 576, 579, 580, 588, 591, 592, 615, 618, 623, 625, 628, 656, 666, 688, 689, 693, 697, 704, 706, 710, 711, 729]


In [42]:
for index,row in url_store_1.iterrows():
    
#     if index == 3:
        
    deep_url = row['row_player_url_deeper']

    country = row['country']

    shirt_number = str(row['shirt_number'])

    index_number = str(row['index_number'])
    
    print(index_number)

    file_save_location = 'player_pages/' + country + '_' + shirt_number  + '_' + index_number + '.html'

    responsex = requests.get(deep_url, headers = headersx)
    
    delay = randint(3,5)
    time.sleep(delay)       

    with open(file_save_location, 'wb') as f:
        f.write(responsex.content)
    
print('done and done')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
66
67
68
69
72
73
74
75
76
77
78
79
80
82
83
87
89
90
92
93
94
95
96
97
98
99
100
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
136
137
138
139
140
141
142
143
144
145
147
148
149
150
151
152
154
155
156
157
158
159
160
161
162
164
166
167
168
169
170
171
172
173
175
176
177
178
179
180
182
183
184
186
187
188
189
192
193
195
196
199
200
201
202
205
206
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
230
231
232
233
234
235
236
237
238
239
240
241
242
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
264
267
270
271
272
273
276
279
280
282
284
285
286
287
288
290
293
294
295
296
301
303
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
3

In [43]:
len(not_there_list)

141

In [45]:
pd.options.display.max_rows

60

In [46]:
pd.options.display.max_rows = 500

In [52]:
not_there_df = player_df[player_df['index'].isin(not_there_list)]

In [53]:
#code for finding age from https://stackoverflow.com/questions/2217488/age-from-birthdate-in-python
today = date.today()

for index,row in not_there_df.iterrows():
    date_birth_raw = row['date_of_birth']
    date_birth = datetime.strptime(date_birth_raw,'%d.%m.%Y')
    age = today.year - date_birth.year - ((today.month, today.day) < (date_birth.month, date_birth.day))
    not_there_df.loc[index,'player_age'] = int(age)

/home/shijith/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/shijith/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [54]:
not_there_df.to_csv('data/byhand_input.csv', index=False)

In [57]:
player_df = pd.read_csv('data/fifa_player_list.csv')

df_with_short_urls = pd.read_csv('data/byhand_input_forwork.csv')

for index,row in df_with_short_urls.iterrows():
    short_url = row['row_player_url'] 
    long_url = short_url.replace("/profil/","/leistungsdatendetails/")
    df_with_short_urls.loc[index,'row_player_url_deeper'] = long_url

df_with_short_urls.to_csv('data/player_urls_remaining.csv', index=False)

In [59]:
remaining_urls_df = pd.read_csv('data/player_urls_remaining.csv')

for index,row in remaining_urls_df.iterrows():
    
#     if index == 3:
        
    deep_url = row['row_player_url_deeper']

    country = row['country']

    shirt_number = str(row['shirt_number'])

    index_number = str(row['index'])
    
    print(index_number)

    file_save_location = 'player_pages/' + country + '_' + shirt_number  + '_' + index_number + '.html'

    responsex = requests.get(deep_url, headers = headersx)
    
    delay = randint(3,5)
    time.sleep(delay)       

    with open(file_save_location, 'wb') as f:
        f.write(responsex.content)
    
print('done and done')

349
117
398
554
568
569
573
574
704
191
207
706
197
553
556
558
559
563
565
566
405
204
557
562
561
567
185
304
579
263
101
70
353
628
266
268
666
688
85
91
283
419
417
366
23
88
228
243
269
427
369
368
163
689
710
580
358
84
275
289
521
729
367
365
281
392
364
656
615
591
174
363
413
347
354
362
396
292
340
537
203
618
407
560
418
693
571
291
194
229
299
544
410
297
86
274
44
403
302
71
265
300
135
277
181
298
81
146
623
625
153
356
576
542
348
357
359
350
165
548
198
592
361
424
65
352
697
351
445
278
355
570
346
360
190
382
588
711
536
541
547
done and done


In [81]:
player_df = pd.read_csv('data/fifa_player_list.csv')

labels = ['index_number','country','shirt_number','fifa_name','date_birth','position','club',\
         'assists','appearances','minutes','appearances_per_assist','minutes_per_assist']

with open("data/player_assists.csv", "w") as filex:   
    wr = csv.writer(filex, delimiter = ',' , quotechar = '"' )
    wr.writerow(labels)

for index,row in player_df.iterrows():
    if index not in [349,361,457]: #these are players for which there is no table on the page

        index_number = str(row['index'])
        country = row['country']
        shirt_number = str(row['shirt_number'])
        fifa_name = row['fifa_display_name']
        date_birth = row['date_of_birth']
        position = row['position']
        club = row['club']

        file_open_location = 'player_pages/' + country + '_' + shirt_number + '_' + index_number + '.html'
        with open(file_open_location) as filex:
            sourcex = filex.read()
            treex = html.document_fromstring(sourcex)
            
#             #check for data table
#             name = treex.xpath("//*[@id='yw1_c0']/a/text()")[0]
#             print(name)
            
#             #check for pages without assist column
#             if ((treex.xpath("//*[@id='yw1_c7']/a/span/@title")[0] != 'Assists') and (position != 'GK')):
#                 print(file_open_location)

            assists = 0
            appearances = 0
            minutes = 0
            
            seasons_allowed = ['16/17','17/18','2016','2017','2018']
            
            if treex.xpath("//*[@id='yw1_c7']/a/span/@title")[0] == 'Assists':
                
                print(index)

                rows = treex.xpath("//*[@id='yw1']/table/tbody//tr")
                for row in rows:
                    season = row.xpath("td[1]/text()")[0]
                    if season in seasons_allowed:
                        season_appearances_raw = row.xpath("td[5]/a/text()")[0]
                        
                        if season_appearances_raw == '-':
                            season_appearances = 0
                        else:
                            season_appearances = int(season_appearances_raw)
                        
                        appearances += season_appearances
                        
                        assists_raw = row.xpath("td[8]/text()")[0]
                        if assists_raw == '-':
                            season_assists = 0
                        else:
                            season_assists = int(assists_raw)
                        
                        assists += season_assists
                        
                        minutes_rawest = row.xpath("td[10]/text()")[0]
                        
                        minutes_raw = minutes_rawest.replace('.','').replace("'",'')
                        if minutes_raw == '-':
                            season_minutes = 0
                        else:
                            season_minutes = int(minutes_raw)
                        
                        minutes += season_minutes
                        
            if assists == 0:
                appearances_per_assist = 0
                minutes_per_assist = 0
            else:
                appearances_per_assist = appearances/assists
                minutes_per_assist = minutes/assists
                        
            new_row = [index_number,country,shirt_number,fifa_name,date_birth,position,club,\
                 assists,appearances,minutes,appearances_per_assist,minutes_per_assist]
            
            with open("data/player_assists.csv", "a") as filex:   
                wr = csv.writer(filex, delimiter = ',' , quotechar = '"' )
                wr.writerow(new_row)

#do a check for messi to see if you got the math right                        
#                         cell_data = treex.xpath("//*[@id='yw1']/table/tbody/tr[1]/td[1]/text()")[0]
#                         print(cell_data)

print('done and done')

1
2
3
4
5
6
7
8
9
10
12
13
14
15
16
17
18
19
20
21
24
25
26
27
28
29
30
31
32
33
35
36
37
38
39
41
42
43
44
45
47
48
49
50
51
52
53
54
55
56
59
60
61
62
63
64
65
66
67
68
70
71
72
73
74
75
76
77
78
79
80
81
82
83
85
86
87
88
89
90
93
94
95
96
97
98
99
100
101
102
104
105
106
107
108
109
110
111
112
114
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
136
139
140
141
142
143
144
145
146
147
148
150
151
152
153
154
155
156
157
158
159
162
163
164
165
166
167
168
169
170
171
172
173
174
175
177
178
179
180
181
183
185
186
187
188
189
190
191
192
193
194
195
196
197
198
200
201
202
203
204
205
208
209
210
211
212
213
214
215
216
217
218
220
221
222
223
224
225
226
227
228
231
232
233
234
235
236
237
238
239
240
241
242
243
244
246
247
248
249
250
251
254
255
256
257
258
259
260
261
262
263
265
266
267
268
269
270
271
272
273
275
277
278
279
280
281
282
283
284
285
286
289
290
291
292
293
294
295
296
297
298
300
301
302
303
304
305
306
307
308
309
311
312
313
314
